<a href="https://colab.research.google.com/github/ashishbhatta/Assignment-2/blob/main/Question_2/2_Fault_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time

from sklearn.model_selection import train_test_split #needed for spliting data into testing and training
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RepeatedKFold

In [ ]:
df_LG = pd.read_csv(r"D:\Notebook\LG.csv")
df_LL = pd.read_csv(r"D:\Notebook\LL.csv")
df_LLG=pd.read_csv(r"D:\Notebook\LLG.csv")
df_LLL = pd.read_csv(r"D:\Notebook\LLL_fault.csv")

df = pd.concat([df_LG, df_LL, df_LLG, df_LLL],ignore_index=True,axis=0) # ignore_index=True,
print(df)

data=np.array(df)
X=data[:,1:4]
y=data[:,4]

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Notebook\\LG.csv'

In [ ]:
#Step 1: Understanding the Data

# Scatter plot of the inputs vs target
plt.scatter(X[:, 0], y, label='I_a vs Fault Type')
#plt.scatter(X[:, 1], y, label='I_b vs Fault Type')
#plt.scatter(X[:, 2], y, label='I_c vs Fault Type')

plt.xlabel('Inputs')
plt.ylabel('Target')
plt.legend()
plt.title('Output vs Input Visualization')
plt.show()

In [ ]:
#Step2: Scaling the data

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.2)
#print("Train data are:")
#print(X_train)
#print(y_train)

#Normalizing the Input
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#Step 3:  Training with various Model

def ML_classifier(classifier,train_inputs= X_train_scaled,train_target = y_train, test_inputs= X_test_scaled, test_output = y_test,model_name = ""):
    t = time.process_time()
    classifier.fit(train_inputs,train_target)
    #training_time = time.process_time() - t
    print(f"Training Time =",time.process_time() - t)
    print("Training Accuracy =" ,accuracy_score(train_target, classifier.predict(train_inputs)))
    print("Training Precision =", precision_score(train_target, classifier.predict(train_inputs), average='micro'))
    print("Training Recall =", recall_score(train_target, classifier.predict(train_inputs), average='micro'))
    print("Training F1 Score =", f1_score(train_target, classifier.predict(train_inputs), average='micro'))
    print("--------------------------------------------------------------------------")
    #t = time.process_time()
    classifier.fit(test_inputs, test_output) #for MLR
    #training_time = time.process_time() - t
    print("Testing Time =",time.process_time() - t)
    print("Testing Accuracy =" ,accuracy_score(test_output, classifier.predict(test_inputs)))
    print("Testing Precision =", precision_score(test_output, classifier.predict(test_inputs), average='micro'))
    print("Testing Recall =", recall_score(y_test, classifier.predict(test_inputs), average='micro'))
    print("Testing F1 Score =", f1_score(y_test, classifier.predict(test_inputs), average='micro'))

    y_pred=classifier.predict(test_inputs);
    y_pred=np.array(y_pred)

    plt.scatter(test_output, y_pred, color='blue', label='Predicted vs Actual')
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')  # Ideal line
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Predicted vs Actual for {model_name} Model')
    plt.legend()
    plt.savefig(model_name + ".svg",)
    plt.show()

    print("--------------------------------------------------------------------------")
    print("--------------------------------------------------------------------------")


In [ ]:
print("logistic Regression Model")
classifier = LogisticRegression();
ML_classifier(classifier,train_inputs= X_train_scaled,train_target = y_train, test_inputs= X_test_scaled, test_output = y_test,model_name = "LR_classifier")

classifier = svm.SVC();
ML_classifier(classifier,train_inputs= X_train_scaled,train_target = y_train, test_inputs= X_test_scaled, test_output = y_test,model_name = "SVM")

classifier = tree.DecisionTreeClassifier()
ML_classifier(classifier,train_inputs= X_train_scaled,train_target = y_train, test_inputs= X_test_scaled, test_output = y_test,model_name = "DecisionTreeClassifier")

classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
ML_classifier(classifier,train_inputs= X_train_scaled,train_target = y_train, test_inputs= X_test_scaled, test_output = y_test,model_name = "ANN_Classifier")


classifier = RandomForestClassifier()
ML_classifier(classifier,train_inputs= X_train_scaled,train_target = y_train, test_inputs= X_test_scaled, test_output = y_test,model_name = "Random_Forest_Classifier")

classifier = XGBClassifier(n_estimators=100, random_state=42)
ML_classifier(classifier,train_inputs= X_train_scaled,train_target = y_train, test_inputs= X_test_scaled, test_output = y_test,model_name = "XGB_classifier")



# Hyper Parameter Tuning for ANN

In [ ]:
#Hyperparameter Tuning for ANN

param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],  # Different layer configurations
    'activation': ['relu', 'tanh', 'logistic'],  # Activation functions
    #'solver': ['adam', 'sgd'],  # Solvers
    #'learning_rate': ['constant', 'invscaling', 'adaptive'],  # Learning rate schedules
    'learning_rate_init': [0.001, 0.01, 0.1]  # Initial learning rate values
}

# Instantiate the classifier
classifier = MLPClassifier(random_state=12345, max_iter=1000)


# Instantiate 5-fold CV repeated 3 times (reduce folds and repeats)
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=54321)


# Instantiate grid search CV
grid_cv = GridSearchCV(classifier, param_grid, scoring='accuracy', n_jobs=5, cv=cv, refit=True)

# Perform grid search
grid_cv.fit(X_train_scaled, y_train)

# Which parameters were the best?
print(f"Best Parameters: {grid_cv.best_params_}")

# Get the best estimator (classifier with best hyperparameters)
best_classifier = grid_cv.best_estimator_

# Evaluate the best model on the test set
y_pred = best_classifier.predict(X_test_scaled)
print(f"Accuracy on Test Set: {accuracy_score(y_test, y_pred)}")